In [1]:
# -*- coding: utf-8 -*-
"""
Adaptado para comparar GT con predicciones de Cellpose
"""

from glob import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tifffile import imread
from cc3d import connected_components
from stardist.matching import matching
from tqdm import tqdm
from skimage.segmentation import clear_border

def normalize(img):
    return (img - img.min()) / (img.max() - img.min())

def leer_imagenes(path):
    names = sorted(glob(os.path.join(path, "*.tif")))
    if not names:
        raise ValueError(f"No se encontraron imágenes TIFF en {path}")
    print(f"Imágenes leídas desde {path}: {len(names)}")
    images = [clear_border(connected_components(imread(name))) for name in names]
    names = [os.path.basename(name).split(".")[0] for name in names]
    return images, names

def evaluate_models(gt_images, preds_dict, names, output_dir=".", thresholds=np.linspace(0.5, 1.0, 11), fmt=['o-']):
    os.makedirs(output_dir, exist_ok=True)
    ap_results = {}
    std_results = {}

    for model_name, pred_images in preds_dict.items():
        print(f"\nEvaluando {model_name}...")
        matches = [
            [matching(gt_images[j], pred_images[j], thresh=i, report_matches=True)
            for i in tqdm(thresholds, desc=f"Thresholds {model_name}")]
            for j in range(len(gt_images))
        ]
        ap_values = [[m.accuracy for m in match] for match in matches]
        ap_results[model_name] = np.mean(ap_values, axis=0)
        std_results[model_name] = np.std(ap_values, axis=0)

    plt.figure(figsize=(10, 6))
    for idx, model_name in enumerate(preds_dict.keys()):
        plt.errorbar(
            thresholds,
            ap_results[model_name],
            yerr=std_results[model_name],
            fmt=fmt[idx % len(fmt)],
            label=model_name,
            capsize=5
        )
    plt.legend()
    plt.ylabel("AP")
    plt.xlabel(r"IoU threshold ($\tau$)")
    plt.title("Detection Scores")
    plt.savefig(os.path.join(output_dir, "detection_scores.png"), dpi=300)
    plt.close()

    thresh = 0.5
    detailed_results = []

    for model_name, pred_images in preds_dict.items():
        print(f"\nEvaluando detalladamente {model_name} con IoU={thresh}...")
        matches = [matching(gt_images[j], pred_images[j], thresh=thresh)
                   for j in tqdm(range(len(gt_images)))]

        keys = ['criterion', 'thresh', 'fp', 'tp', 'fn', 'precision', 'recall',
                'accuracy', 'f1', 'n_true', 'n_pred', 'mean_true_score',
                'mean_matched_score', 'panoptic_quality']
        df = pd.DataFrame([m._asdict() for m in matches], columns=keys)
        df.insert(0, 'Image', names)
        df.insert(0, 'Model', model_name)
        detailed_results.append(df)

    df_results = pd.concat(detailed_results, ignore_index=True)

    float_cols = ['precision', 'recall', 'accuracy', 'f1', 'mean_true_score', 'mean_matched_score', 'panoptic_quality']
    int_cols = ['fp', 'tp', 'fn', 'n_true', 'n_pred']

    df_results[float_cols] = df_results[float_cols].astype(float).round(4)
    df_results[int_cols] = df_results[int_cols].astype(int)

    new_names = {
        "fp": "FP", "tp": "TP", "fn": "FN",
        "precision": "Precision", "recall": "Recall",
        "accuracy": "Average Precision", "f1": "F1-Score",
        "n_true": "N True", "n_pred": "N Pred"
    }
    df_results = df_results.rename(columns=new_names)

    df_results['FP (%)'] = (df_results['FP'] / df_results['N Pred']).round(4)
    df_results['TP (%)'] = (df_results['TP'] / df_results['N Pred']).round(4)
    df_results['FN (%)'] = (df_results['FN'] / df_results['N Pred']).round(4)

    metrics = ['FP (%)', 'TP (%)', 'FN (%)', 'Precision', 'Recall', 'Average Precision', 'F1-Score']
    df_mean = df_results.groupby('Model', sort=False)[metrics].mean()
    df_std = df_results.groupby('Model', sort=False)[metrics].std()

    df_results.to_csv(os.path.join(output_dir, "detailed_results.csv"), sep=";", index=False)
    df_mean.to_csv(os.path.join(output_dir, "mean_results.csv"), sep=";")
    df_std.to_csv(os.path.join(output_dir, "std_results.csv"), sep=";")

    return df_results, df_mean, df_std

if __name__ == "__main__":
    gt_path = r"\\WS1\WS1_Remote_Disk\Current Segovia Lab\Emilio Gutiérrez\Emilio Gutiérrez SOS\test Cellpose\test set\GT"
    pred_path = r"\\WS1\WS1_Remote_Disk\Current Segovia Lab\Emilio Gutiérrez\Emilio Gutiérrez SOS\test Cellpose\test set\masks"
    out_dir = r"\\WS1\WS1_Remote_Disk\Current Segovia Lab\Emilio Gutiérrez\Emilio Gutiérrez SOS\test Cellpose\test set\results plots"

    gt, names = leer_imagenes(gt_path)
    preds_dict = {"CellPose": leer_imagenes(pred_path)[0]}

    results, mean_results, std_results = evaluate_models(
        gt,
        preds_dict,
        names,
        output_dir=out_dir,
        thresholds=np.linspace(0.5, 1.0, 11),
        fmt=['o-']
    )

    print("\nResultados promedio:")
    print(mean_results)

ModuleNotFoundError: No module named 'seaborn'

In [2]:
pip install seaborn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [1]:
# -*- coding: utf-8 -*-
"""
Adaptado para comparar GT con predicciones de Cellpose
"""

from glob import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tifffile import imread
from cc3d import connected_components
from stardist.matching import matching
from tqdm import tqdm
from skimage.segmentation import clear_border

def normalize(img):
    return (img - img.min()) / (img.max() - img.min())

def leer_imagenes(path):
    names = sorted(glob(os.path.join(path, "*.tif")))
    if not names:
        raise ValueError(f"No se encontraron imágenes TIFF en {path}")
    print(f"Imágenes leídas desde {path}: {len(names)}")
    images = [clear_border(connected_components(imread(name))) for name in names]
    names = [os.path.basename(name).split(".")[0] for name in names]
    return images, names

def evaluate_models(gt_images, preds_dict, names, output_dir=".", thresholds=np.linspace(0.5, 1.0, 11), fmt=['o-']):
    os.makedirs(output_dir, exist_ok=True)
    ap_results = {}
    std_results = {}

    for model_name, pred_images in preds_dict.items():
        print(f"\nEvaluando {model_name}...")
        matches = [
            [matching(gt_images[j], pred_images[j], thresh=i, report_matches=True)
            for i in tqdm(thresholds, desc=f"Thresholds {model_name}")]
            for j in range(len(gt_images))
        ]
        ap_values = [[m.accuracy for m in match] for match in matches]
        ap_results[model_name] = np.mean(ap_values, axis=0)
        std_results[model_name] = np.std(ap_values, axis=0)

    plt.figure(figsize=(10, 6))
    for idx, model_name in enumerate(preds_dict.keys()):
        plt.errorbar(
            thresholds,
            ap_results[model_name],
            yerr=std_results[model_name],
            fmt=fmt[idx % len(fmt)],
            label=model_name,
            capsize=5
        )
    plt.legend()
    plt.ylabel("AP")
    plt.xlabel(r"IoU threshold ($\tau$)")
    plt.title("Detection Scores")
    plt.savefig(os.path.join(output_dir, "detection_scores.png"), dpi=300)
    plt.close()

    thresh = 0.5
    detailed_results = []

    for model_name, pred_images in preds_dict.items():
        print(f"\nEvaluando detalladamente {model_name} con IoU={thresh}...")
        matches = [matching(gt_images[j], pred_images[j], thresh=thresh)
                   for j in tqdm(range(len(gt_images)))]

        keys = ['criterion', 'thresh', 'fp', 'tp', 'fn', 'precision', 'recall',
                'accuracy', 'f1', 'n_true', 'n_pred', 'mean_true_score',
                'mean_matched_score', 'panoptic_quality']
        df = pd.DataFrame([m._asdict() for m in matches], columns=keys)
        df.insert(0, 'Image', names)
        df.insert(0, 'Model', model_name)
        detailed_results.append(df)

    df_results = pd.concat(detailed_results, ignore_index=True)

    float_cols = ['precision', 'recall', 'accuracy', 'f1', 'mean_true_score', 'mean_matched_score', 'panoptic_quality']
    int_cols = ['fp', 'tp', 'fn', 'n_true', 'n_pred']

    df_results[float_cols] = df_results[float_cols].astype(float).round(4)
    df_results[int_cols] = df_results[int_cols].astype(int)

    new_names = {
        "fp": "FP", "tp": "TP", "fn": "FN",
        "precision": "Precision", "recall": "Recall",
        "accuracy": "Average Precision", "f1": "F1-Score",
        "n_true": "N True", "n_pred": "N Pred"
    }
    df_results = df_results.rename(columns=new_names)

    df_results['FP (%)'] = (df_results['FP'] / df_results['N Pred']).round(4)
    df_results['TP (%)'] = (df_results['TP'] / df_results['N Pred']).round(4)
    df_results['FN (%)'] = (df_results['FN'] / df_results['N Pred']).round(4)

    metrics = ['FP (%)', 'TP (%)', 'FN (%)', 'Precision', 'Recall', 'Average Precision', 'F1-Score']
    df_mean = df_results.groupby('Model', sort=False)[metrics].mean()
    df_std = df_results.groupby('Model', sort=False)[metrics].std()

    df_results.to_csv(os.path.join(output_dir, "detailed_results.csv"), sep=";", index=False)
    df_mean.to_csv(os.path.join(output_dir, "mean_results.csv"), sep=";")
    df_std.to_csv(os.path.join(output_dir, "std_results.csv"), sep=";")

    return df_results, df_mean, df_std

if __name__ == "__main__":
    gt_path = r"\\WS1\WS1_Remote_Disk\Current Segovia Lab\Emilio Gutiérrez\Emilio Gutiérrez SOS\test Cellpose\test set\GT"
    pred_path = r"\\WS1\WS1_Remote_Disk\Current Segovia Lab\Emilio Gutiérrez\Emilio Gutiérrez SOS\test Cellpose\test set\masks"
    out_dir = r"\\WS1\WS1_Remote_Disk\Current Segovia Lab\Emilio Gutiérrez\Emilio Gutiérrez SOS\test Cellpose\test set\results plots"

    gt, names = leer_imagenes(gt_path)
    preds_dict = {"CellPose": leer_imagenes(pred_path)[0]}

    results, mean_results, std_results = evaluate_models(
        gt,
        preds_dict,
        names,
        output_dir=out_dir,
        thresholds=np.linspace(0.5, 1.0, 11),
        fmt=['o-']
    )

    print("\nResultados promedio:")
    print(mean_results)

ModuleNotFoundError: No module named 'cc3d'

In [2]:
# -*- coding: utf-8 -*-
"""
Adaptado para comparar GT con predicciones de Cellpose
"""

from glob import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Asegurar que todas las dependencias estén instaladas automáticamente
required_packages = [
    ('seaborn', 'seaborn'),
    ('tifffile', 'tifffile'),
    ('cc3d', 'cc3d'),
    ('stardist', 'stardist'),
    ('tqdm', 'tqdm'),
    ('skimage', 'scikit-image')
]

import importlib
import subprocess
import sys

for module_name, package_name in required_packages:
    try:
        importlib.import_module(module_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

import seaborn as sns
from tifffile import imread
from cc3d import connected_components
from stardist.matching import matching
from tqdm import tqdm
from skimage.segmentation import clear_border

def normalize(img):
    return (img - img.min()) / (img.max() - img.min())

def leer_imagenes(path):
    names = sorted(glob(os.path.join(path, "*.tif")))
    if not names:
        raise ValueError(f"No se encontraron imágenes TIFF en {path}")
    print(f"Imágenes leídas desde {path}: {len(names)}")
    images = [clear_border(connected_components(imread(name))) for name in names]
    names = [os.path.basename(name).split(".")[0] for name in names]
    return images, names

def evaluate_models(gt_images, preds_dict, names, output_dir=".", thresholds=np.linspace(0.5, 1.0, 11), fmt=['o-']):
    os.makedirs(output_dir, exist_ok=True)
    ap_results = {}
    std_results = {}

    for model_name, pred_images in preds_dict.items():
        print(f"\nEvaluando {model_name}...")
        matches = [
            [matching(gt_images[j], pred_images[j], thresh=i, report_matches=True)
            for i in tqdm(thresholds, desc=f"Thresholds {model_name}")]
            for j in range(len(gt_images))
        ]
        ap_values = [[m.accuracy for m in match] for match in matches]
        ap_results[model_name] = np.mean(ap_values, axis=0)
        std_results[model_name] = np.std(ap_values, axis=0)

    plt.figure(figsize=(10, 6))
    for idx, model_name in enumerate(preds_dict.keys()):
        plt.errorbar(
            thresholds,
            ap_results[model_name],
            yerr=std_results[model_name],
            fmt=fmt[idx % len(fmt)],
            label=model_name,
            capsize=5
        )
    plt.legend()
    plt.ylabel("AP")
    plt.xlabel(r"IoU threshold ($\tau$)")
    plt.title("Detection Scores")
    plt.savefig(os.path.join(output_dir, "detection_scores.png"), dpi=300)
    plt.close()

    thresh = 0.5
    detailed_results = []

    for model_name, pred_images in preds_dict.items():
        print(f"\nEvaluando detalladamente {model_name} con IoU={thresh}...")
        matches = [matching(gt_images[j], pred_images[j], thresh=thresh)
                   for j in tqdm(range(len(gt_images)))]

        keys = ['criterion', 'thresh', 'fp', 'tp', 'fn', 'precision', 'recall',
                'accuracy', 'f1', 'n_true', 'n_pred', 'mean_true_score',
                'mean_matched_score', 'panoptic_quality']
        df = pd.DataFrame([m._asdict() for m in matches], columns=keys)
        df.insert(0, 'Image', names)
        df.insert(0, 'Model', model_name)
        detailed_results.append(df)

    df_results = pd.concat(detailed_results, ignore_index=True)

    float_cols = ['precision', 'recall', 'accuracy', 'f1', 'mean_true_score', 'mean_matched_score', 'panoptic_quality']
    int_cols = ['fp', 'tp', 'fn', 'n_true', 'n_pred']

    df_results[float_cols] = df_results[float_cols].astype(float).round(4)
    df_results[int_cols] = df_results[int_cols].astype(int)

    new_names = {
        "fp": "FP", "tp": "TP", "fn": "FN",
        "precision": "Precision", "recall": "Recall",
        "accuracy": "Average Precision", "f1": "F1-Score",
        "n_true": "N True", "n_pred": "N Pred"
    }
    df_results = df_results.rename(columns=new_names)

    df_results['FP (%)'] = (df_results['FP'] / df_results['N Pred']).round(4)
    df_results['TP (%)'] = (df_results['TP'] / df_results['N Pred']).round(4)
    df_results['FN (%)'] = (df_results['FN'] / df_results['N Pred']).round(4)

    metrics = ['FP (%)', 'TP (%)', 'FN (%)', 'Precision', 'Recall', 'Average Precision', 'F1-Score']
    df_mean = df_results.groupby('Model', sort=False)[metrics].mean()
    df_std = df_results.groupby('Model', sort=False)[metrics].std()

    df_results.to_csv(os.path.join(output_dir, "detailed_results.csv"), sep=";", index=False)
    df_mean.to_csv(os.path.join(output_dir, "mean_results.csv"), sep=";")
    df_std.to_csv(os.path.join(output_dir, "std_results.csv"), sep=";")

    return df_results, df_mean, df_std

if __name__ == "__main__":
    gt_path = r"\\WS1\WS1_Remote_Disk\Current Segovia Lab\Emilio Gutiérrez\Emilio Gutiérrez SOS\test Cellpose\test set\GT"
    pred_path = r"\\WS1\WS1_Remote_Disk\Current Segovia Lab\Emilio Gutiérrez\Emilio Gutiérrez SOS\test Cellpose\test set\masks"
    out_dir = r"\\WS1\WS1_Remote_Disk\Current Segovia Lab\Emilio Gutiérrez\Emilio Gutiérrez SOS\test Cellpose\test set\results plots"

    gt, names = leer_imagenes(gt_path)
    preds_dict = {"CellPose": leer_imagenes(pred_path)[0]}

    results, mean_results, std_results = evaluate_models(
        gt,
        preds_dict,
        names,
        output_dir=out_dir,
        thresholds=np.linspace(0.5, 1.0, 11),
        fmt=['o-']
    )

    print("\nResultados promedio:")
    print(mean_results)

CalledProcessError: Command '['C:\\Users\\WORKSTATION-PC\\anaconda3\\envs\\selfnet\\python.exe', '-m', 'pip', 'install', 'cc3d']' returned non-zero exit status 1.

In [3]:
# -*- coding: utf-8 -*-
"""
Adaptado para comparar GT con predicciones de Cellpose
"""

from glob import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Asegurar que todas las dependencias estén instaladas automáticamente, excepto cc3d
required_packages = [
    ('seaborn', 'seaborn'),
    ('tifffile', 'tifffile'),
    ('stardist', 'stardist'),
    ('tqdm', 'tqdm'),
    ('skimage', 'scikit-image')
]

import importlib
import subprocess
import sys

for module_name, package_name in required_packages:
    try:
        importlib.import_module(module_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

# Intentar importar cc3d manualmente
try:
    from cc3d import connected_components
except ModuleNotFoundError:
    raise ImportError("El paquete 'cc3d' no está instalado. Instálalo manualmente con:\nconda install -c conda-forge cc3d")

import seaborn as sns
from tifffile import imread
from stardist.matching import matching
from tqdm import tqdm
from skimage.segmentation import clear_border

def normalize(img):
    return (img - img.min()) / (img.max() - img.min())

def leer_imagenes(path):
    names = sorted(glob(os.path.join(path, "*.tif")))
    if not names:
        raise ValueError(f"No se encontraron imágenes TIFF en {path}")
    print(f"Imágenes leídas desde {path}: {len(names)}")
    images = [clear_border(connected_components(imread(name))) for name in names]
    names = [os.path.basename(name).split(".")[0] for name in names]
    return images, names

def evaluate_models(gt_images, preds_dict, names, output_dir=".", thresholds=np.linspace(0.5, 1.0, 11), fmt=['o-']):
    os.makedirs(output_dir, exist_ok=True)
    ap_results = {}
    std_results = {}

    for model_name, pred_images in preds_dict.items():
        print(f"\nEvaluando {model_name}...")
        matches = [
            [matching(gt_images[j], pred_images[j], thresh=i, report_matches=True)
            for i in tqdm(thresholds, desc=f"Thresholds {model_name}")]
            for j in range(len(gt_images))
        ]
        ap_values = [[m.accuracy for m in match] for match in matches]
        ap_results[model_name] = np.mean(ap_values, axis=0)
        std_results[model_name] = np.std(ap_values, axis=0)

    plt.figure(figsize=(10, 6))
    for idx, model_name in enumerate(preds_dict.keys()):
        plt.errorbar(
            thresholds,
            ap_results[model_name],
            yerr=std_results[model_name],
            fmt=fmt[idx % len(fmt)],
            label=model_name,
            capsize=5
        )
    plt.legend()
    plt.ylabel("AP")
    plt.xlabel(r"IoU threshold ($\\tau$)")
    plt.title("Detection Scores")
    plt.savefig(os.path.join(output_dir, "detection_scores.png"), dpi=300)
    plt.close()

    thresh = 0.5
    detailed_results = []

    for model_name, pred_images in preds_dict.items():
        print(f"\nEvaluando detalladamente {model_name} con IoU={thresh}...")
        matches = [matching(gt_images[j], pred_images[j], thresh=thresh)
                   for j in tqdm(range(len(gt_images)))]

        keys = ['criterion', 'thresh', 'fp', 'tp', 'fn', 'precision', 'recall',
                'accuracy', 'f1', 'n_true', 'n_pred', 'mean_true_score',
                'mean_matched_score', 'panoptic_quality']
        df = pd.DataFrame([m._asdict() for m in matches], columns=keys)
        df.insert(0, 'Image', names)
        df.insert(0, 'Model', model_name)
        detailed_results.append(df)

    df_results = pd.concat(detailed_results, ignore_index=True)

    float_cols = ['precision', 'recall', 'accuracy', 'f1', 'mean_true_score', 'mean_matched_score', 'panoptic_quality']
    int_cols = ['fp', 'tp', 'fn', 'n_true', 'n_pred']

    df_results[float_cols] = df_results[float_cols].astype(float).round(4)
    df_results[int_cols] = df_results[int_cols].astype(int)

    new_names = {
        "fp": "FP", "tp": "TP", "fn": "FN",
        "precision": "Precision", "recall": "Recall",
        "accuracy": "Average Precision", "f1": "F1-Score",
        "n_true": "N True", "n_pred": "N Pred"
    }
    df_results = df_results.rename(columns=new_names)

    df_results['FP (%)'] = (df_results['FP'] / df_results['N Pred']).round(4)
    df_results['TP (%)'] = (df_results['TP'] / df_results['N Pred']).round(4)
    df_results['FN (%)'] = (df_results['FN'] / df_results['N Pred']).round(4)

    metrics = ['FP (%)', 'TP (%)', 'FN (%)', 'Precision', 'Recall', 'Average Precision', 'F1-Score']
    df_mean = df_results.groupby('Model', sort=False)[metrics].mean()
    df_std = df_results.groupby('Model', sort=False)[metrics].std()

    df_results.to_csv(os.path.join(output_dir, "detailed_results.csv"), sep=";", index=False)
    df_mean.to_csv(os.path.join(output_dir, "mean_results.csv"), sep=";")
    df_std.to_csv(os.path.join(output_dir, "std_results.csv"), sep=";")

    return df_results, df_mean, df_std

if __name__ == "__main__":
    gt_path = r"\\WS1\WS1_Remote_Disk\Current Segovia Lab\Emilio Gutiérrez\Emilio Gutiérrez SOS\test Cellpose\test set\GT"
    pred_path = r"\\WS1\WS1_Remote_Disk\Current Segovia Lab\Emilio Gutiérrez\Emilio Gutiérrez SOS\test Cellpose\test set\masks"
    out_dir = r"\\WS1\WS1_Remote_Disk\Current Segovia Lab\Emilio Gutiérrez\Emilio Gutiérrez SOS\test Cellpose\test set\results plots"

    gt, names = leer_imagenes(gt_path)
    preds_dict = {"CellPose": leer_imagenes(pred_path)[0]}

    results, mean_results, std_results = evaluate_models(
        gt,
        preds_dict,
        names,
        output_dir=out_dir,
        thresholds=np.linspace(0.5, 1.0, 11),
        fmt=['o-']
    )

    print("\nResultados promedio:")
    print(mean_results)

ImportError: El paquete 'cc3d' no está instalado. Instálalo manualmente con:
conda install -c conda-forge cc3d

In [4]:
# -*- coding: utf-8 -*-
"""
Adaptado para comparar GT con predicciones de Cellpose
"""

from glob import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Lista de paquetes para intentar instalar automáticamente si no están presentes
required_packages = [
    ('seaborn', 'seaborn'),
    ('tifffile', 'tifffile'),
    ('stardist', 'stardist'),
    ('tqdm', 'tqdm'),
    ('skimage', 'scikit-image')
]

import importlib
import subprocess
import sys

# Solo instalar paquetes que se sabe funcionan bien vía pip
for module_name, package_name in required_packages:
    try:
        importlib.import_module(module_name)
    except ModuleNotFoundError:
        print(f"Instalando {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])

# Verificar si cc3d está instalado
try:
    from cc3d import connected_components
except ModuleNotFoundError:
    raise ImportError(
        "\n\n❌ El paquete 'cc3d' no está instalado.\n"
        "Instálalo manualmente en tu entorno con:\n"
        "conda install -c conda-forge cc3d\n"
    )

import seaborn as sns
from tifffile import imread
from stardist.matching import matching
from tqdm import tqdm
from skimage.segmentation import clear_border

def normalize(img):
    return (img - img.min()) / (img.max() - img.min())

def leer_imagenes(path):
    names = sorted(glob(os.path.join(path, "*.tif")))
    if not names:
        raise Valu


ImportError: 

❌ El paquete 'cc3d' no está instalado.
Instálalo manualmente en tu entorno con:
conda install -c conda-forge cc3d
